## ELT
- 대시 보드에 사용할 분석용 테이블

1. 최근 6개월 내 상영되고 있는 영화 장르 현황
2. 장르별 누적 관객수가 높은 영화(만 단위)
3. 장르별 누적 매출액이 높은 영화(억 단위)

In [ ]:
-- DROP TABLE
-- 기존 테이블을 아예 삭제시키고 다시 CREATE TABLE을 해야하는 경우에 사용
/*
DROP TABLE IF EXISTS analytics.recent_movie_genres;
DROP TABLE IF EXISTS analytics.genre_cumulative_audience;
DROP TABLE IF EXISTS analytics.genre_cumulative_sales;
*/

### 최근 6개월 내 상영되고 있는 영화 장르 현황

In [ ]:
CREATE TABLE analytics.recent_movie_genres AS 
SELECT A.genres, count_table.movie_count AS cnt
FROM raw_data.movie_genre A
JOIN raw_data.movie_sale B ON A.movieCd = B.movieCd
JOIN (
    SELECT genres, COUNT(DISTINCT A.moviecd) AS movie_count
    FROM raw_data.movie_genre A
    JOIN raw_data.movie_sale B ON A.movieCd = B.movieCd
    WHERE DATEDIFF(dget B.date, GETDATE()) < 180
    GROUP BY genres
) count_table ON A.genres = count_table.genres
WHERE DATEDIFF(day, B.date, GETDATE()) < 180
GROUP BY A.genres, count_table.movie_count
ORDER BY cnt DESC;

In [ ]:
select * from analytics.recent_movie_genres

### 장르별 누적 관객수가 높은 영화(만 단위)

In [ ]:
CREATE TABLE analytics.genre_cumulative_audience AS (
  WITH genre_cumulative_audience AS (
    SELECT A.genres, 
        FLOOR(B.audiAcc / 10000) AS audiAcc, 
        B.movieNm,
        ROW_NUMBER() OVER (PARTITION BY A.genres ORDER BY B.audiAcc DESC) AS rn
    FROM raw_data.movie_genre A
    JOIN raw_data.movie_sale B ON A.movieCd = B.movieCd
  )
  SELECT genres, movieNm, audiAcc
  FROM genre_cumulative_audience
  WHERE rn = 1
);

In [ ]:
select * from analytics.genre_cumulative_audience
order by genres

### 장르별 누적 매출액이 높은 영화(억 단위)

In [ ]:
CREATE TABLE analytics.genre_cumulative_sales AS (
    WITH genre_cumulative_sales AS (
        SELECT A.genres,
        FLOOR(B.salesAcc / 100000000) AS salesAcc,
        B.movieNm,
        ROW_NUMBER() OVER (PARTITION BY A.genres ORDER BY B.salesAcc DESC) AS rn
    FROM raw_data.movie_genre A
    JOIN raw_data.movie_sale B ON A.movieCd = B.movieCd
)
    SELECT genres, movieNm, salesAcc
    FROM genre_cumulative_sales
    WHERE rn = 1
);

In [ ]:
select * from analytics.genre_cumulative_sales